In [ ]:
import numpy as np
import pandas as pd
import albumentations
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import PIL
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


np.random.seed(42)





from matplotlib import style


DATA = 'ARTS'
# DATA = "GTSRB"
device = 'cuda'


PATH_TO_IMAGE = "<PATH_TO_IMAGE_FILE>"

image_size = 60
IMG_HEIGHT = 60
IMG_WIDTH = 60

NUM_CATEGORIES = 43

class GtsrbDataset(Dataset):
    def __init__(self, df, mode, transform=None):

        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        self.paths = df['path'].values
        self.labels = df['label'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img = np.array(PIL.Image.open(self.paths[index]).convert('RGB'))
        if self.transform is not None:
            res = self.transform(image=img)
            img = res['image']

        img = img.transpose(2, 0, 1)
        label = self.labels[index]
        return img, label


transforms_valid = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
    albumentations.Normalize()
])

#valid_dataset = GtsrbDataset(df_train, 'valid', transform=transforms_valid)

map_models = {}

from torch import nn

class SimpleCNN(nn.Module):
    def __init__(self, out_dim):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(128),
            
            nn.Flatten(),
            nn.Linear(18432, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.5),
            nn.Linear(512, out_dim),
        )

    def forward(self, x):
        logits = self.net(x)
        return logits


# load classifier
model = SimpleCNN(NUM_CATEGORIES)
kernel_type = 'SimpleCNN'

data = torch.load(f'models_60/SimpleCNN_{DATA}_best_loss.pth', map_location=torch.device('cpu'))
model.load_state_dict(data['model'])
    
classes_GTSRB = data['classes']
model = model.to(device).eval()
map_models['SimpleCNN'] = model, classes_GTSRB
# STOPSIGN = 14 # GTSRB

STOPSIGN = 12 # ARTS
from datetime import datetime

class GtsrbDataset(Dataset):
    def __init__(self, df, mode, transform=None):

        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        self.paths = df['path'].values
        self.labels = df['label'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img = np.array(PIL.Image.open(self.paths[index]).convert('RGB'))
        if self.transform is not None:
            res = self.transform(image=img)
            img = res['image']

        img = img.transpose(2, 0, 1)
        label = self.labels[index]
        return img, label


transforms_valid = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
    albumentations.Normalize()
])


INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [ ]:
path = PATH_TO_IMAGE
import cv2
image = cv2.imread(path)
at_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

res = transforms_valid(image=at_image)
img = res['image']
img = torch.tensor(img.transpose(2, 0, 1))#.unsqueeze(0)
img = torch.stack([img], axis=0).to(device)
map_res = {}

# text = '/'.join(PATH_TO_IMAGE.split('/')[-2:])
text = '/'.join(path.split('/')[-2:])
name_ = str(text)

all_res = []
for k, (model, classes_GTSRB) in map_models.items():
    pred = model(img).softmax(axis=1).cpu().detach().numpy()
    label = pred.argmax(axis=1)[0]

    text += f'\n{DATA} {k}: {classes_GTSRB[label]} ({pred.max():.2f})' # model_name,class,(prob)
    prob_of_stopsign = pred[0][STOPSIGN]
    prob = pred.max()
    map_res = dict(name=name_, label=classes_GTSRB[label], prob=pred.max(), model=k)

    all_res.append(map_res)

print(text) 
print(prob_of_stopsign)
plt.imshow(at_image)

[ WARN:0@4.285] global loadsave.cpp:268 findDecoder imread_('crp2.png'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
